In [1]:
%load_ext autoreload
%autoreload 2

from os import listdir
from os.path import isfile, join

import numpy as np
import matplotlib.pyplot as plt
import madmom

import sys
sys.path.append('../src')
from preprocessing import get_dataset
from models import OLSPatchRegressor

na = np.newaxis

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [4]:
music_dir  = '../data/music_speech/music_wav/'
speech_dir = '../data/music_speech/speech_wav/'

max_samples = -1

X, Y = get_dataset(music_dir, speech_dir, num_samples=max_samples, hpool=0, wpool=0)
print('Train Set Shape')
print(X.shape, Y.shape)

Y = (Y + 1) / 2 

Train Set Shape
(128, 74, 1500, 1) (128,)


# Train CNN using KERAS

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [51]:
batch_size = 8
num_classes = 2
epochs = 109

# --------------------------------------------
num_frequencies = X.shape[1]
num_timesteps   = X.shape[2]
num_channels    = X.shape[3]
input_shape = num_frequencies, num_timesteps, num_channels

# DEFINE MODEL
model = Sequential()
model.add(MaxPooling2D(pool_size=(1, 3), input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(num_frequencies, 3),
                 activation='relu'))

model.add(Conv2D(1, kernel_size=(1, 1), activation='sigmoid'))
model.add(Lambda(lambda x: K.mean(x, axis=[1,2])))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [50]:
t_ind = 96
XTrain = X[:t_ind]
YTrain = Y[:t_ind]
XTest  = X[t_ind:]
YTest  = Y[t_ind:]

print(XTrain.shape, YTrain.shape)
print(XTest.shape, YTest.shape)

(96, 74, 1500, 1) (96,)
(32, 74, 1500, 1) (32,)


In [52]:
model.fit(XTrain, YTrain,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(XTest, YTest))
score = model.evaluate(XTest, YTest, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 96 samples, validate on 32 samples
Epoch 1/109
96/96 [==============================] - 1s 6ms/step - loss: 0.6936 - acc: 0.4479 - val_loss: 0.6928 - val_acc: 0.3438
Epoch 2/109
96/96 [==============================] - 0s 4ms/step - loss: 0.6697 - acc: 0.5208 - val_loss: 0.7056 - val_acc: 0.4062
Epoch 3/109
96/96 [==============================] - 0s 4ms/step - loss: 0.6658 - acc: 0.5521 - val_loss: 0.6999 - val_acc: 0.3438
Epoch 4/109
96/96 [==============================] - 0s 4ms/step - loss: 0.6423 - acc: 0.5625 - val_loss: 0.7063 - val_acc: 0.3438
Epoch 5/109
96/96 [==============================] - 0s 4ms/step - loss: 0.6284 - acc: 0.5833 - val_loss: 0.7174 - val_acc: 0.3438
Epoch 6/109
96/96 [==============================] - 0s 4ms/step - loss: 0.6279 - acc: 0.5833 - val_loss: 0.7002 - val_acc: 0.5938
Epoch 7/109
96/96 [==============================] - 0s 4ms/step - loss: 0.6234 - acc: 0.6875 - val_loss: 0.7201 - val_acc: 0.3125
Epoch 8/109
96/96 [====================